# AI Game Playing Algorithms with Checkers

Ryan Williams, Ben Newell, Chris Haynes

## Introduction

## Board Class
### Overview
This class contains the entire functionality of checkers. It handles everything required to turn the game of checkers into a trainable game for the different AI algorithms. The functions that we have learned throughout the semester that are quintessential to many algorithms, such as `stateMoveTuple`, `validMoves`, etc. all reside in the board class. Since the class is very large, the functions will be described in the order they appear to make it easier to understand. 

### Functions

#### Constructor: `__init__(self)`
This function initializes the game of checkers. It uses an 8x8 numpy array called `draught` (the official name of the checkers board) of `Piece` objects to initialize the board. If a piece is absent, the space is set to `None`. Initial positions are the first three and last three rows and alternate between a left alignment and right alignment. Since the color red starts the game, the turn is set to `Color.RED`, where color is an enumeration. This could have been simplified to a boolean saying `red_turn = True` for red's turn and `red_turn = False` for black's turn. However for readability of my teammates I added the enumeration.

#### Representation: `__repr__(self)`
This function converts a board to a string so it is easier to print and make `stateMoveTuples`. It creates a format string that represents a row, and for every row add the `Piece` string representation and if it is `None` add a dash to represent the empty space. It has indexed columns and rows that correspond to the indices of the numpy array `draught`.

#### Print State: `printState(self)`
This function prints a board, which calls the `__repr__` function described earlier.

#### State Move Tuple: `stateMoveTuple(self, move)`
This function takes a move as input and returns a tuple with the state of the board as a string and a move (which is already a tuple).

#### Make Move: `makeMove(self, move)`
This function takes a move and will make a move on the board. It uses the length of the move tuple to determine if any jumps occur during the move. If a jump is made, the pieces that were jumped during the move are set to `None` in the `draught`. It will then set the pieces previous space to `None` and update the space that it is moving to with moved `Piece`. If a `Piece` has reached the opponents edge of the `draught`, the `Piece` will be promoted to a king. The turn is then changed.

#### Change Turn: `changeTurn(self)`
This changes the turn of the board using a ternary operator to determine which turn to set the `Board` to the correct turn.

#### Valid Moves: `validMoves(self)`
This function determines the list of valid moves for a `Board` based on the turn. It begins by filtering for the `valid_pieces` on the `Board`. An array is initialized to hold all `valid_moves`. For every piece in the `valid_pieces` it checks if it can move left or right based on its color if it is not a king. So black can move up in rows and red can move down. Kings can move in all directions. This uses the `checkSpace` helper function to determine legal moves in that direction. It filters out the moves returned and adds it to the array it returns in the tuple format of `(initial position, move)`. For multiple jumps, they are recorded in the move. 

#### Check Space: `checkSpace(self, position, row_mod, col_mod, king=False, recurse=False, positions=None)`
This function takes as input an initial position, directions to move, whether or not the piece is a king, and recursion arguments. It starts by getting the `new_row` and `new_col` for a space to check. If it is in bounds, a check is performed to see if it is unoccupied. If it is, and it's not recursing (which happens during a jump), it will return that position only since it is an unoccupied space. If it is occupied and can be jumped and hasn't already been added to a jump pattern, it will check for extra jumps by checking the left and right positions again recursively. For kings it checks three positions instead of four (in `validMoves` it uses four) because a piece may not be jumped twice. If more jumps are found, they are added to the move. Return the list of valid moves.

#### Can Be Jumped: `canBeJumped(self, attacker, defender)`
Checks if a piece can be jumped. It takes as input an attacking position and a defending position. The massive if statement determines the direction of the attack and if the piece has no piece on the other side of a jump, it may be jumped and will return `True`. In the case that it cannot be jumped, it will return `False`. 

# Piece Class
## Overview
This class represents a `Piece` on the `Board` in the `Board.draught` array. The class is fairly simple, but contains data critical to a `Piece`'s functionality in the game.

## Functions

#### Constructor: `__init__(self, color, initial_position)`
The constructor take as input the `Color` of the `Piece`, and it's `initial_position` on the board, which is a tuple in the form of (row, column). It is initialized as a normal piece rather than a king, so `king = False`. 

#### Representation: `__repr__(self)`
This is the string function for a `Piece`. It is the `Color` of the piece as a character ('B' for `Color.BLACK` and 'R' for `Color.RED`. If it is a king, a 'K' is appended to the string. So a red king is represented as 'RK' instead of just 'R'.

#### King Me: `king_me(self)`
Promotes a `Piece` to king by setting `Piece.king` to `True`. 

# Reinforcement 
## Overview
Notice that this is not a class like `Board` and `Piece` but rather just functions. These functions will both train and use a `Q` dictionary that stores the a `Board` state move tuple as the key to a number that represents the potential game outcome of making that particular move for that exact scenario. The higher the number, the better that move is for the AI. Our reinforcement is a little bit different than the reinforcement learning we saw in class. This algorithm back propagates a reinforcement throughout every move that led to that state, so that moves early in the game still recieve a value. Also, we significantly reduce the size of the `Q` table by storing just moves, rather than a state move tuple. Checkers has 500 billion billion possible states, not including state move tuples. We aslo did this because moves hold interesting information about the game, such as jumping another piece, getting kinged, etc. due to the way they are represented in the board class, and so that AI can extrapolate the usefulness of the move based on the outcome of the game.

## Functions

#### Epsilon Greedy: `epsilonGreedy(epsilon, Q, board)`
This function selects a move for a board based on an epsilon to add randomness into the selection. It takes an `epsilon` and will choose either a random move if a randomly generated number is less than the `epsilon`. The process is that a list of valid moves is generated for a `Board`. If the random route is selected, the a random move from `valid_moves` is selected. Otherwise, the greedy move for the `Q` dictionary is selected and returned.

#### Greedy: `greedy(valid_moves, Q, board)`
This function will make the greedy move for a board, or basically the move with the most utility as determined by the `Q` dictionary. For every move in `valid_moves`, if the state move tuple is in the dictionary, get its determined utility. Otherwise, it will be 0, a neautral utility because its usefulness has not yet been determined. It will then return the move with the highest utility. If all moves have the same value (most likely zero at the beginning), the first valid move will be chosen. For red this is the first move for the piece with the lowest row first, and lowest column second.

#### Finished: `finished(board)`
Determines if a game is finished. If the length of valid moves from the `board` is zero, the game is over.

#### Train Q: `trainQ(nRepititions, learningRate, epsilonDecayFactor, propagationDecayFactor)`
This function trains a `Q` dictionary for the number `nRepititions` games. The `learningRate` determines how fast the `Q` dictionary will learn for a certain move and board state. The `epsilonDecayFactor` determines the rate at which the `epsilon` passed to `epsilonGreedy` will decay, meaning using the utilities stored in the `Q` more and more. Every game is a new game, in which the AI is set to play as red. For every game, if the game is finished, update the dictionary with a reinforcement. If it is a win, that move will be represented by a 1. If it is a loss it will be -2. We selected -2 since it is playing a random opponent, and losing to a random move maker should be punished harsher than winning against that random player. After a move has been made, it will back propagate that move's utility throughout every move made that led up to it by calling `backPropagateReinforcement`.

#### Back Propagate Reinforcement: `backPropagateReinforcement(Q, move_tuple_list, learningRate, propagationDecayFactor)`
This function will back propagate a reinforcement through the Q dictionary. The reason we added this is there were too many zeros in the Q dictionary, and thus only first moves were selected a decent amount of the time as a result of np.argmax returning the first valid move when a move doesn't have a reinforcement. It back propagates a reinforcement throughout every move made that led to the final board state, so that every move has a reinforcement (every move matters!).

#### Use Q: `useQ(Q, maxSteps)`
This function tests the `Q` dictionary that results from the `trainQ` function. It uses the same game logic as `trainQ`. Red starts the game (red is the reinforcement trained AI). It makes a greedy move by calling `greedy` to determine the optimal move. Then black makes a random move and the game continues until one has lost. 

## Observations
There were some interesting observations along the way that led to structuring our reinforcement AI the way we did. First of all, we saw that using state move tuples worked just fine and the AI would beat the random with state move tuples as the keys to the `Q` dictionary. However, upon inspection there were way too many states and `Q` was very sparse after training, with often only values for less than 10% of all entries. 

The first method we tried to solve this was to back propagate the reinforcement with a very low decay rate so that the reinforcement would reach the opening move. However, these states were never used because there were too many to ever get a hit on the table. So how was that AI winning? Well, `numpy.argmax` in the `greedy` function takes the first valid move, so the strategy was inadvertantly to move the first piece possible up and to the left if it could, and never to move the last row. Since black could never move into the last row and make a king, it runs out of valid moves and the AI would win. 

To try and reduce this strategy and allow `Q` to actually contain some semblance of a strategy, we reduced `Q` to only contain the move selected. The move has inherent state involved, such as where it is moving from, the spaces it will jump, and the end position. This seemed to be enough to allow the AI to still ~98-99% of the time against a random opponent. This way we could look at the Q dictionary and see the values for the moves it selected. It still used the strategy of avoiding moving out of the back row if it could, but at times would jump a piece that had gotten close. It also really valued moving up the board as much as possible to try and get a king. If these moves were available they were often chosen, which could be a residual impact of the `numpy.argmax` usage we discussed earlier. Finally, it's winning strategy after observing many games. It tries it's best to get as many kings as possible, and leaves it's back row completely full, thus blocking the opponent from making any kings. Once it has kings and black has no kings, the opponent will inevitably run out of moves, and the AI will win the game.

#### Further Possible Research
Some interesting thoughts we had along the way and after discussing was using a smart test subject to train against. When looking at the moves made, the AI will take risks because a random opponent will rarely select the right move that would punish that risk, such as moving up the board and being vulnerable for a jump. It has one thing in mind: getting a king. To try and prevent foolish behavior such as this, a smarter training opponent could be added that would punish the AI by jumping it when it could. 

Another interesting idea is to use a neural net to determine the reinforcements. Possible reinforcements could be jumping pieces, getting kings, favoring the sides as opposed to the middle, and not moving off the back wall. If weights were determined through a neaural net for these reinforcements, the AI could be much smarter, and produce more accurate reinforcements for certain moves. 

Finally, we thought that it would be interesting to represent the keys to the `Q` dictionary in different ways. One could include a localized board section, where selecting the locally optimum would hopefully result in the global optimum. Another could be recognizing similar board patterns and using those patterns with move types (normal, jump, king, etc) as inputs into the dictionary. The point of changing how keys are represented is to reduce the number of states, while still maintaining a decently accurate `Q` dictionary.

## Adversarial Search

Checkers is a zero-sum game, meaning the payoff for each player at the end of the game is equal and opposite. For this reason, adversarial search algorithms like `minimax` and `negamax` can be used to play the game of checkers. Through a process similar to a depth first search, the algorithms search through each possible move from the initial player. Then, from each of these moves, the algorithm searches each move from there for the opposing player. Once the depth limit or leaf nodes are reached, the utility is returned and the player chooses the move that optimizes their utility. From there, in the next recursion up, that player chooses it's optimum move and so on. This process results in one of the key components of adversarial search algorithms, each player assumming the other is playing optimally. Both of the two adversarial search algorithms, `minimax` and `negamax` follow this same process. The way they differ is in how players' utility is determined, which will be explained below.

### Necessary funtions added to Board class

#### isOver

`isOver` determines if the game is over, and if it is, gives the winner. It does this by checking the valid moves for each player. If the player has no valid moves, they have lost and the other player is the winner. This can be done by taking all of the opposing pieces (no pieces mean no valid moves) or by trapping all of the opponents pieces so they cannot move. If both players have valid moves they can do, the game is not over yet so the function returns `False` for the result and `None` for the winner.

isOver(self)

Parameters:
* None

Return:
* boolean - true if game is over, false otherwise
* winner - RED or BLACK depending on the winner. If game is not over, value is None

#### getUtility

`getUtility` gets the utility for the calling player at the specific point in the game. If type is `nm` and the game is over, the function will return 2 if the calling player won and -2 if the calling player lost. If type is `mm`, however, the function will return 2 or -2 if the calling player won, 2 if it's the maximizer, -2 if it's the minimizer. If the calling player lost, -2 is returned if the caller is the maximizer and 2 is returned if the calling player is the minimizer.

If the game is not over, then a heuristic is used to determine utility. The number of red and black pieces are counted with kings counting as two. If the type is `nm`, Whichever color has more pieces is awarded a utility of 1 and the color with less pieces receives a utility of -1 . Of course, if the type is `mm`, then the utilities are slightly different, with 1 going to maximizer if it has more pieces and -1 going to minimizer if it has more pieces. Therefore, the maximizer will get a utility of -1 if it has less pieces and the minimizer will get 1 if it has less. Finally, if the number of pieces is equal, a utility of 0 is returned.

The utility function was made in this way so that winning supercedes all. At first, the thought was to use 1 as the utility for both winning and having more pieces, but we decided that it makes the most sense for the algorithm to choose the winning move if there is an option between that and merely having more pieces. The most simple way to do this was to set winning utility as greater than the heuristic so it would always be chosen.

getUtility(self, type='nm', maximizePlayer=Color.RED)

Parameters:
* type: 'mm' or 'nm' depending on whether the algorithm is `minimax` or `negamax` since they give different utilities to each player
* maximizePlayer: Color of the maximizing player

Return:
* integer - represents the utility given at this stage of the game

### Minimax

#### Algorithm

The two players in the minimax algorithm are `max` and `min`. `Max` will look to maximize the utility and `min` will look to minimize the utility. For our case, in our checkers implementation, a win for `max`is worth 2 and a win for `min` is worth -2. `Minimax` searches through every possible move in the game until the depth limit, assuming on the way that the opponent will play optimally. If the depth limit is reached, a simple heuristic is used to see who has the advantage. All of the pieces for each player will be counted, with kings counting as two. If `max` has more, a utility of 1 will be awarded; if `min` has more, however, a utility of -1 is awarded. 

One thing added to this algorithm to cut down run time was a `break` statement used if the returned utility is 2 for the maximizer or -2 for minimizer. This was done because these values signify a win for their respective player so there is no reason to search any other nodes on that level of the tree. Even though a win could be achieved with a different move yet to be searched from that state, the algorithm wouldn't even choose that path because the returned value would equal the `bestValue` but not be greater than it, so `bestValue` would not be updated anyway.

#### Alpha beta pruning

Alpha beta pruning is a technique used in adversarial search algorithms to limit the number subtrees that need to be searched. It does this by taking the best value found so far for one player, represented by the variable `alpha`, and comparing it to the best value found so far for the other player, represented by the variable `beta`. If `alpha` is ever greater than `beta`, all other subtrees at that level do not need to be searched and can be pruned because, even if they were to be searched, they would not be chosen by the algorithm. To explain further, even though the current node may have chosen a value in the pruned subtree, the node above the current is guarenteed to not choose anything in that subtree since it has already found a better value in a previous subtree. As a result, cutting off these subrees saves a lot of time.

The implementation of alpha beta pruning is simple for `minimax`. First, The parameters `alpha` and `beta` are added to the funtion with `alpha` always representing the best value for the maximizer and `beta` always representing the best value for the minimizer. Inside the function, if the current player is the maximizer, after a value and a move is returned by the recursive call to `minimax`, the current best value found is compared to `alpha`. If `bestValue` is greater than `alpha`, `alpha` will be set to `bestValue`. If the current player is the minimzer, however, the current best value is compared to `beta` and if `bestValue` is less than `beta`, `beta` is set to `bestValue`. Finally, following this process for both the maximizer and minimizer, `alpha` is compared to `beta` and if `alpha` is greater than `beta`, a `break` statement is used and no more of the moves for that specific state are checked.

#### Function definition

minimax(board, depthLeft, alpha, beta, isMax=True)

Parameters:
* `board` : board object with necessary functions like validMoves, makeMove, etc.
* `depthLeft` : depth left before depth limit is reached and heuristic is used.
* `alpah` : alpha value for alpha beta pruning, represents best value so far for the maximizing player
* `beta` : beta value for alpha beta pruning, represents best value so far for the minimizing player
* `isMax` : deliniates whether this player is the maximizing player. 

Return:
* best value found and the resulting best move

### Negamax

#### Algorithm

As opposed to `minimax`, both players are looking to maximize their utility. As a result, a win for either player yields a utility of 2 and if the depth limit is reached, the heuristic is used giving either player a utility of one if they have more pieces than the other player. The algorithm flips the sign of the returned value from the recursive call, allowing both sides to look to maximize utility. Like the `minimax` algorithm, `negamax` also assumes the opponent will play optimally. 

Similar to `minimax`, a `break` statement is used if the utility is ever two, representing a win. The logic behind this operation is the same as the logic in `minimax`. If a win is found there is no reason to search any other moves because nothing better will be found, and, based on the way the algorithm updates the `bestValue`, once a win is discovered, another win will not result in `bestValue` being update.

#### Alpha beta pruning

Alpha beta pruning works very similarly in `negamax` to the way it does in `minimax`. `alpha` and `beta` are still added to the function definition as parameters, but one difference is `alpha` is the best value so far for the current player and `beta` is the best value so far for the opposing player. Additionally, in the recursive call to the negamax function, `-beta` is passed in as the `alpha` parameter and `-alpha` is passed in as the `beta` parameter. This is done so, inside the recursive call, the `alpha` and `beta` values correctly represent the best so far values for their respective players. Despite the differences, the function of alpha beta pruning remains the same for `negamax` as it is for `minimax`. If `alpha` is ever greater than `beta`, all of the subtrees at that level can be pruned because they would never be chosen by the algorithm.

#### Function definition

negamax(board, depthLeft, alpha, beta)

Parameters:
* `board` : board object with necessary functions like validMoves, makeMove, etc.
* `depthLeft` : depth left before depth limit is reached and heuristic is used.
* `alpah` : alpha value for alpha beta pruning, represents best value so far for the maximizing player
* `beta` : beta value for alpha beta pruning, represents best value so far for the minimizing player

Return:
* best value found and the resulting best move

### Examples

#### Minimax

In [6]:
def minimaxExample1():
    # Setup
    board = Board()
    piece = Piece(Color.RED, [3, 2])
    piece2 = Piece(Color.BLACK, [2, 1])
    draught = np.empty(shape=(8, 8), dtype=object)
    draught[3, 2] = piece
    draught[2, 1] = piece2
    board.setBoard(draught)
    board.printState()

    value, move = minimax(board, 9, -inf, inf)
    board.makeMove(move)
    print(value)
    board.printState()

In [ ]:
def minimaxExample2():
    board = Board()
    piece = Piece(Color.RED, [3, 4])
    piece4 = Piece(Color.RED, [3, 2])
    piece5 = Piece(Color.RED, [2, 5])
    piece2 = Piece(Color.BLACK, [2, 3])
    piece3 = Piece(Color.BLACK, [0, 3])
    draught = np.empty(shape=(8, 8), dtype=object)
    draught[3, 4] = piece
    draught[3, 2] = piece4
    draught[2, 5] = piece5
    draught[2, 3] = piece2
    draught[0, 3] = piece3
    board.setBoard(draught)
    board.printState()

    isMax = True
    isOver, _ = board.isOver()
    while not isOver:
        value, move = minimax(board, 10, -inf, inf, isMax)
        print("move: ", move)
        if move is None:
            print('move is None. Stopping')
            break
        print("\nPlayer", board.turn, "to", move, "for value", value)
        board.makeMove(move)
        print(board)
        isMax = not isMax
        isOver, _ = board.isOver()

In [ ]:
def minimaxExample3():
    board = Board()
    board.printState()

    isMax = True
    isOver, _ = board.isOver()
    while not isOver:
        if board.turn == Color.BLACK:
            move = board.validMoves()[int(len(board.validMoves()) / 2)]
            print("\nPlayer", board.turn, "to", move)
            board.makeMove(move)
        else:
            value, move = minimax(board, 10, -inf, inf, isMax)
            print("move: ", move)
            if move is None:
                print('move is None. Stopping')
                break
            print("\nPlayer", board.turn, "to", move, "for value", value)
            board.makeMove(move)
        print(board)
        isMax = not isMax
        isOver, _ = board.isOver()

#### Negamax

In [4]:
def negamaxExample1():
    # Setup
    board = Board()
    piece = Piece(Color.RED, [3, 2])
    piece2 = Piece(Color.BLACK, [2, 1])
    draught = np.empty(shape=(8, 8), dtype=object)
    draught[3, 2] = piece
    draught[2, 1] = piece2
    board.setBoard(draught)
    board.printState()

    value, move = negamax(board, 9, -inf, inf)
    board.makeMove(move)
    print(value)
    board.printState()

In [ ]:
def negamaxExample2():
    # Setup
    board = Board()
    piece = Piece(Color.RED, [3, 4])
    piece4 = Piece(Color.RED, [3, 2])
    piece5 = Piece(Color.RED, [2, 5])
    piece2 = Piece(Color.BLACK, [2, 3])
    piece3 = Piece(Color.BLACK, [0, 3])
    draught = np.empty(shape=(8, 8), dtype=object)
    draught[3, 4] = piece
    draught[3, 2] = piece4
    draught[2, 5] = piece5
    draught[2, 3] = piece2
    draught[0, 3] = piece3
    board.setBoard(draught)
    board.printState()

    isOver, _ = board.isOver()
    while not isOver:
        value, move = negamax(board, 5, -inf, inf)
        print("move: ", move)
        if move is None:
            print('move is None. Stopping')
            break
        print("\nPlayer", board.turn, "to", move, "for value", value)
        board.makeMove(move)
        print(board)
        isOver, _ = board.isOver()

In [ ]:
def negamaxExample3():
    board = Board()
    board.printState()

    isOver, _ = board.isOver()
    while not isOver:
        if board.turn == Color.BLACK:
            move = board.validMoves()[int(len(board.validMoves()) / 2)]
            print("\nPlayer", board.turn, "to", move)
            board.makeMove(move)
        else:
            value, move = negamax(board, 10, -inf, inf)
            print("move: ", move)
            if move is None:
                print('move is None. Stopping')
                break
            print("\nPlayer", board.turn, "to", move, "for value", value)
            board.makeMove(move)
        print(board)
        isOver, _ = board.isOver()

### Observations for Adversarial Search

First of all, time is a serious issue for these adversarial search algorithms. Without alpha beta pruning they run for an incredibly long time in the order of hours. As a result, alpha beta pruning is essentially a requirement for these algorithms as it cuts down the runtime immensely. However, even with the addition of alpha beta pruning, time is still a big issue for these adversarial search algorithms. Running an entire game against an opponent who chooses random moves still takes around 30 minutes, since most moves for each player need to be searched and there are 10<sup>20</sup>. 

Setting a depth limit and heuristic is also important for this problem since it saves a lot of time. However, the use of the depth limit could potentially lead to a move that isn't optimal being chosen, but the heuristic does its best in the attempt to choose what it sees as the best move once the depth limit is reached. Additionally, the depth limit must be chosen wisely. A lower depth limit will result in faster run time but a higher depth limit allows the algorithm to search deeper into the game, therefore giving it a higher likelyhood of choosing a good move. 

This leads to an issue encountered in the application of adversarial search to checkers. If the depth limit is low, when it is reached not enough has happened yet in the game for the heuristic to decide who has an advantage, and a utility of 0 is returned since both sides have an even amount of pieces. The utility being 0 happens for much of the first half of the game before a player is deemed to have an advantage, and, while bad moves are still avoided, the algorithm stll chooses the first of potentially many moves with a 0 utility possibly missing an optimum move.

Another issue observed due to the heuristic was sometimes a player seemed to decide to capture another piece just to get the piece count to be equal or in their favor, even if that capture is not the best move to do. As described earlier, the first half of the game is spent with the utility being 0 since both sides have an equal piece count. One potential reason for this is a player choosing to capture a piece to achieve the 0 utility when it would have a -1 utility (1 if the player is the minimizer in minimax) by not capturing. When looking at the board at these times, the capture seems illogical, but is still chosen because of the piece count reason. This same phenomenon is observed closer to the end of the game when an advantage can be derived. Seemingly illogical captures still take place just so the player will still have a greater piece count. However, even though these moves seem illogical from the onset because they lead to immediate capture of the origional piece or merely result in a perceived disadvantage, maybe prioritzing capturing and piece count is a worthwile stategy to consider in a checkers game.

### Further Research and Improvements

There is some further research that could be done for these adversarial search algorithms, most of which relating to speed. One of the biggest things that could be improved is creating a better heuristic. With a improved heuristic that can determine advantage earlier in the game with more levels of utilities, a wider range of values will be found earlier on. This would inevetably result in more pruning of subtrees since there will be more than just the five utility values of -2, -1, 0, 1, and 2 to compare. Additionally, the heuristic could be improved allowing for a prioritization of certain types of moves. For example, capturing or kinging could be prioritized if the utility would otherwise be the same. Even not moving certain pieces could be prioritized like not moving pieces in the back row. Statistics keeping on certain moves could also be done in an attempt to remember what moves are good, in a manner similar to reinforcement learning. This way, with knowledge of the statistics, certain moves could be chosen over others if the utility is otherwise the same.

In [3]:
import io
from nbformat import current
import glob
nbfile = glob.glob('Williams-Newell-Haynes-Final-Project.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Williams-Newell-Haynes-Final-Project.ipynb is 2330


## Overall Observations